In [1]:
import os
import pandas as pd
import numpy as np
pathresults = "../../scripts/results/"
#pathresults = "/DUMUXDune27/DUMUX/dumux-rosi/python/paperSc/results/"
data_file_delimiter = ","

def getVal(mypath, filename, header="infer",names = None):
    fullpath = pathresults +mypath+filename
    if names is None:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header).dropna(how= "all", axis=1)
    else:
        cst = pd.read_csv(fullpath,delimiter=data_file_delimiter, header = header, names = names).dropna(how= "all", axis=1)
    return cst


def get_last_line_as_numpy_array(file_path, filename,dtype = float):
    with open(pathresults+file_path+filename, 'r') as file:
        # Read all lines
        lines = file.readlines()

        # Check if there are any lines in the file
        if not lines:
            raise ValueError("The file is empty")

        # Get the last line
        last_line = lines[-1].strip()

        # Split the last line by commas and convert to NumPy array
        array_from_last_line = np.array(last_line.split(','), dtype=dtype)

        return array_from_last_line

In [8]:
def getprogress(p2f,setsI):
    tt = getVal(p2f, "time.txt",names= ['time','Qlight'])
    time = tt.iloc[:,0]
    wrsi =  np.array(getVal(p2f, "inner_error.csv")[['errWrsi','errWrsiRealInput']].iloc[-1] )
    errs = []
    for nc in range(9):
        errs.append(int(max(get_last_line_as_numpy_array(p2f, "content_diff1d3drel"+str(nc)+".csv"))*1e5, 
                             )/1e5)
    timeStep = getVal(p2f, "inner_loopdata.csv")[['dt_inner', 'n_iter']].iloc[-1] 
    timeStep['dt_inner'] *= 24 *60 #*60
    errsbulk = get_last_line_as_numpy_array(p2f, "errorsBulkSoilCumulative.csv")#[[4,5]]
    realTime = getVal(p2f, "totalComputetime.txt",names= ['tot','V1','V2','V3'])['tot'].iloc[-1] /3600
    print(setsI , '{:.2f}'.format(max(time)),'\t',
         np.array(timeStep*100,dtype=int)/100, 'realTime', '{:.2f}'.format(realTime),
          np.array(wrsi*100,dtype=int)/100,errs,
           r'mean speed: {:.2f}'.format(max(time-10)/realTime),
          "\n\t\t\t",r'errsbulk {:.0e},{:.0e}'.format(errsbulk[0],errsbulk[1]),
          "\n\t", get_last_line_as_numpy_array(p2f, "errorsBulkSoilCumulative.csv")
             )

In [9]:
path2file =r'/TraiRhizo/paperSc/NewSetup/1cm0{}_1440_{}_10to25_20mn_0s_32/'
scenarios = ["baseline","earlyDry", "lateDry"] 
setIds = [5,44,61]

setsAll = [(scsc, idid) for scsc in scenarios for idid in setIds ]

for i, setsI  in enumerate(setsAll):
    scenario =setsI[0]
    setId=str(setsI[1])
    p2f = path2file.format( scenario, setId)
    getprogress(p2f,setsI)

('baseline', 5) 25.00 	 [19.99  4.  ] realTime 8.71 [0. 0.] [0.0, 0.00102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 1.72 
			 errsbulk 1e-15,4e-13 
	 [1.44328993e-15 3.71140088e-13 0.00000000e+00 0.00000000e+00]
('baseline', 44) 25.00 	 [19.99  4.  ] realTime 8.71 [0. 0.] [0.0, 0.00018, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 1.72 
			 errsbulk 7e-16,2e-13 
	 [6.66133815e-16 1.52872095e-13 0.00000000e+00 0.00000000e+00]
('baseline', 61) 25.00 	 [19.99  4.  ] realTime 8.68 [0. 0.] [0.0, 0.00038, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 1.73 
			 errsbulk 2e-15,5e-13 
	 [2.27595720e-15 5.41687436e-13 0.00000000e+00 0.00000000e+00]
('earlyDry', 5) 25.00 	 [9.99 4.  ] realTime 10.48 [0. 0.] [0.0, 0.00272, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 1.43 
			 errsbulk 9e-16,2e-13 
	 [8.88178420e-16 2.31106682e-13 0.00000000e+00 0.00000000e+00]
('earlyDry', 44) 25.00 	 [9.99 4.  ] realTime 10.80 [0. 0.] [0.0, 0.00563, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 1.39 

In [49]:
path2file = r'testspeed/thr{}{}_1476_{}_10to25_20mn_0s_32/'
scenarios = ["baseline","earlyDry", "lateDry"] 
setIds = [5,44,61]

nthrds = [2,4,8]

numPset = len(nthrds)
for i, nthrd  in enumerate(nthrds):
    scenario ="lateDry"
    setId="5"
    numthr = nthrd
    p2f = path2file.format(numthr, scenario, setId)
    if True:#try:
        tt = getVal(p2f, "time.txt",names= ['time','Qlight'])
        time = tt.iloc[:,0]
        wrsi =  np.array(getVal(p2f, "inner_error.csv")[['errWrsi','errWrsiRealInput']].iloc[-1] )
        errs = []
        for nc in range(9):
            errs.append(int(max(get_last_line_as_numpy_array(p2f, "content_diff1d3drel"+str(nc)+".csv"))*1000)/1000)
        timeStep = getVal(p2f, "inner_loopdata.csv")[['dt_inner', 'n_iter']].iloc[-1] 
        timeStep['dt_inner'] *= 24 *60 #*60
        realTime = getVal(p2f, "totalComputetime.txt",names= ['tot','V1','V2','V3'])['tot'].iloc[-1] /3600
        print(numthr ,'\t', (int((max(time))*100)/100),'\t',# errs,
             np.array(timeStep*100,dtype=int)/100, 'realTime',int(realTime*100)/100 #,np.array(wrsi*100,dtype=int)/100
              , 'mean speed', max(time-10)/realTime
             )
print()
setsAll = [(scsc, idid) for scsc in scenarios for idid in setIds ]

for i, setsI  in enumerate(setsAll):
    scenario =setsI[0]
    setId=str(setsI[1])
    numthr = str(4)
    p2f = path2file.format(numthr, scenario, setId)
    if True:#try:
        tt = getVal(p2f, "time.txt",names= ['time','Qlight'])
        time = tt.iloc[:,0]
        wrsi =  np.array(getVal(p2f, "inner_error.csv")[['errWrsi','errWrsiRealInput']].iloc[-1] )
        errs = []
        for nc in range(9):
            errs.append(int(max(get_last_line_as_numpy_array(p2f, "content_diff1d3drel"+str(nc)+".csv"))*1000)/1000)
        timeStep_ = getVal(p2f, "inner_loopdata.csv")[['dt_inner', 'n_iter']]
        timeStepdtmin = min(timeStep_['dt_inner'].to_numpy()) *24 *60 
        timeStep= timeStep_.iloc[-1] 
        timeStep['dt_inner'] *= 24 *60 #*60
        realTime = getVal(p2f, "totalComputetime.txt",names= ['tot','V1','V2','V3'])['tot'].iloc[-1] /3600
        print(setsI ,'\t', '{:.2f}'.format(max(time)),'\t',
             np.array(timeStep*100,dtype=int)/100, 'realTime', '{:.2f}'.format(realTime),
              np.array(wrsi*100,dtype=int)/100,errs,
               r'mean speed: {:.2f}'.format(max(time-10)/realTime), r'min dt {:.2f}'.format(timeStepdtmin)
             )

2 	 24.86 	 [9.99 1.  ] realTime 69.28 mean speed 0.21448618367240654
4 	 25.0 	 [9.99 4.  ] realTime 68.19 mean speed 0.21996673933577218
8 	 24.86 	 [ 9.99 14.  ] realTime 69.28 mean speed 0.21448145359284979

('baseline', 5) 	 25.00 	 [19.99  4.  ] realTime 19.78 [0. 0.] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 0.76 min dt 20.00
('baseline', 44) 	 25.00 	 [19.99  4.  ] realTime 19.77 [0. 0.] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 0.76 min dt 20.00
('baseline', 61) 	 25.00 	 [19.99  4.  ] realTime 19.76 [0. 0.] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 0.76 min dt 20.00
('earlyDry', 5) 	 25.00 	 [9.99 4.  ] realTime 44.99 [0. 0.] [0.0, 0.003, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 0.33 min dt 10.00
('earlyDry', 44) 	 25.00 	 [9.99 4.  ] realTime 37.25 [0. 0.] [0.0, 0.004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] mean speed: 0.40 min dt 10.00
('earlyDry', 61) 	 25.00 	 [9.99 4.  ] realTime 41.60 [0. 0.] [0.0, 0.003, 0.0, 0.0, 0.0, 